Новая структура парсера.

1. Получаем ссылки на города.
    1. Можно получить информацию с помощью гет запроса. (менее 1 мни/город)
    2. Требуется куки браузера что бы пройти авторизацию (получается при заходе на страницу через браузер)
2. Получаем информацию о магазинах
    1. get "location_info" выдает информацию об id ключах магазинов. (менее 1 мни/город)
    2. Так же требуется куки браузера
3. Получаем информацию об категориях и субкатегориях
    1. Пока единственный способ - это использовать Selenium. (1-2 час/город)
    2. Необходимо узнать как еще можно получить эту информацию из запросов.
4. Получаем информацию о товарах и скидке через Get запросы.
    1. get "item" выдает информацию об скидках и товарах (15-30 мин/город).
    2. Максимальное количество товара в запросе 600 единиц,
    3. Так же 600 единиц это максимальное количество товара, которое можно ВООБЩЕ получить.
    4. Что бы обойти это ограничение и требуется максимальная детализация в категориях/субкатегория
5. Получение Полной информации о товаре.
    1. в очереди

In [7]:
import pickle
from typing import NamedTuple
import pandas as pd
from urllib.parse import urlparse
from urllib.parse import parse_qs
import numpy as np
import requests
from requests.exceptions import ConnectTimeout, ChunkedEncodingError, ConnectionError
# from tqdm import tqdm
from tqdm.notebook import tqdm
import sys
from multiprocessing.dummy import Pool as ThreadPool
from time import sleep as time_sleep
from pprint import pprint
tqdm.pandas()
requests.packages.urllib3.disable_warnings()

In [8]:
def _save_df_to_zip(df_: pd.DataFrame, archive_name: str = 'archive', folder: str='data', replace: bool=False) -> None:
    # Путь к файлу
    file_path = Path(folder).joinpath(archive_name + '.zip')
    Path(folder).mkdir(exist_ok=True)
    # Проверяем, существует ли файл
    if file_path.exists() and not replace:
        # Получаем время создания файла
        time = datetime.fromtimestamp(file_path.lstat().st_atime).strftime('%Y-%m-%d %H-%M')

        # Создаем новое имя файла с добавлением времени Unix
        new_file_name = file_path.stem + "_" + str(time) + file_path.suffix

        # Создаем новый путь для переименованного файла
        new_file_path = file_path.with_name(new_file_name)
        # Переименовываем файл
        file_path.rename(new_file_path)

# to csv
    compression_opts = dict(method='zip', archive_name=f'{archive_name}.csv')
    df_.to_csv(f'{folder}/{archive_name}.zip', index=False, compression=compression_opts, encoding='utf-8')

In [6]:
seg_df = pd.read_csv('data/segments.zip')
ret_df = pd.read_csv('data/retailers.zip')
loc_df = pd.read_csv('data/located_list.zip')

## Запрос

In [9]:
def get_response(url, params, headers):
    for _ in range(5):
        try:
            respons = requests.get(url, params=params, headers=headers, timeout=(10, 30), verify=False)
        except ConnectTimeout:
            time_sleep(1)
            continue
        except ChunkedEncodingError:
            time_sleep(1)
            continue
        except ConnectionError:
            time_sleep(1)
            continue
        except Exception:
            exc_type, exc_obj, exc_tb = sys.exc_info()
            print(exc_type.__name__)
            return None
        else:    
            return respons.json().get('items', None)
    
    else:
        return None


def get_data(pool_input: tuple[tqdm, tuple[int, str]]):
    pbar, (index, item) = pool_input
    
    out_list = list()
    
    res = loc_df[loc_df['slug'] == item.sity].min()
    headers = {
                'x-locality-geoid': str(res.geoId),
                'x-position-latitude': f'{res.lat:.5f}',
                'x-position-longitude': f'{res.lng:.5f}'
                }
        
    url = f'https://search.edadeal.io/api/v4/retailer/{item.data_uuid}/items'

    params = {  'addContent': ['true'],
                'checkAdult': ['true'],
                'excludeSegmentSlug': ['alcohol', 'pt_alcool', 'en_alcohol', 'es_alcohol', 'tr_alkol'],
                'groupBy': ['sku_or_meta'],
                'numdoc': ['599'],
                'page': ['0'],
                'segmentUuid': []
             }

    # for su1 in tqdm(seg_df['uuid'].unique()[:], desc=f'{item.sity} / {item.market}', leave=False):
    for su1 in seg_df['uuid'].unique()[:]:
        params['segmentUuid'] = [su1]
        
        respons_1 = get_response(url, params, headers)
        
        if respons_1:
            tmp_list_01 = list()
            
            try:
                su2_list = list(set([(x.get('segmentUuids', []))[-1] for x in respons_1]))
                len_sku_list = len(su2_list)
            except Exception:
                su2_list = None
            
            if not su2_list or (len_sku_list>550):
                su2_list = seg_df[seg_df['uuid'] == su1]['uuid level 02'].unique()
            
            for su2 in su2_list:
                params['segmentUuid'] = [su2]

                respons_2 = get_response(url, params, headers)
                                
                if respons_2:
                    tmp_list_02 = list()
                    try:
                        su3_list = list(set([(x.get('segmentUuids', []))[-1] for x in respons_2]))
                        len_sku_list = len(su3_list)
                    except Exception:
                        su3_list = None
                    
                    if not su3_list or (len_sku_list>550):
                        su3_list = seg_df[(seg_df['uuid'] == su1) & (seg_df['uuid level 02'] == su2)]['uuid level 03'].unique()
                    
                    for su3 in su3_list:
                        params['segmentUuid'] = [su3]
                        respons_3 = get_response(url, params, headers)

                        if respons_3:
                            tmp_list_02.append((su3, respons_3))
                    
                    if tmp_list_02:
                        tmp_list_01.extend(tmp_list_02)
                    else:
                        tmp_list_01.append((su2, respons_2))

            if tmp_list_01:
                out_list.extend(tmp_list_01)
            else:
                out_list.append((su1, respons_1))
        else:
            continue
            
    pbar.update(1)
    return (item, out_list)

def run_get_price(list_input, cores: int=2):
    with tqdm(total=len(list_input) ) as pbar:
        list_to_pool = list(zip(([pbar]*len(list_input)), list_input))
        with ThreadPool(cores) as pool:
            work_return = pool.map(get_data, list_to_pool)
    return work_return

In [10]:
def separation_frame(input_df: pd.DataFrame, sep: int=5) -> list[pd.DataFrame]:
    out_list = list()
    chunk_size = input_df.shape[0] // int(100/sep)
    for index in range(0, input_df.shape[0], chunk_size):
        out_list.append(input_df.iloc[index:index+chunk_size])
    return out_list

def convert_to_df(out_list):
    df = pd.DataFrame()
    
    for market, sity in out[:]:
        for uuid, items in sity[:]:
            for sku in items[:]:
                tmp_df = pd.DataFrame({
                                        'title': [sku['title']],
                                        'sku_id': [sku.get('uuid', np.nan)],
                                        'dateStart': [pd.to_datetime(sku.get('dateStart', np.nan), unit='ms')],
                                        'dateEnd': [pd.to_datetime(sku.get('dateEnd', np.nan), unit='ms')],
                                        'price': [sku.get('priceData', {}).get('new', {}).get('from', np.nan)],
                                        'price_from': [sku.get('priceData', {}).get('new', {}).get('to', np.nan)],
                                        'price_to': [sku.get('priceData', {}).get('new', {}).get('value', np.nan)],
                                        'discountPercent': [sku.get('discountPercent', np.nan)],
                                        'quantity': [sku.get('quantity', np.nan)],
                                        'quantityUnit': [sku.get('quantityUnit', np.nan)]
                })
                tmp_df = pd.concat((market.to_frame().T, tmp_df), axis=1)
                tmp_df.insert(7, 'su', uuid)
            
                df = pd.concat((df, tmp_df), axis=0, ignore_index=True)
    return df

def _worker_convert_to_df(pool_input):
    df = pd.DataFrame()
    market, sity = pool_input
    for uuid, items in sity[:]:
        for sku in items[:]:
            tmp_df = pd.DataFrame({
                                    'title': [sku['title']],
                                    'sku_id': [sku.get('uuid', np.nan)],
                                    'dateStart': [pd.to_datetime(sku.get('dateStart', np.nan), unit='ms')],
                                    'dateEnd': [pd.to_datetime(sku.get('dateEnd', np.nan), unit='ms')],
                                    'price': [sku.get('priceData', {}).get('new', {}).get('from', np.nan)],
                                    'price_from': [sku.get('priceData', {}).get('new', {}).get('to', np.nan)],
                                    'price_to': [sku.get('priceData', {}).get('new', {}).get('value', np.nan)],
                                    'discountPercent': [sku.get('discountPercent', np.nan)],
                                    'quantity': [sku.get('quantity', np.nan)],
                                    'quantityUnit': [sku.get('quantityUnit', np.nan)]
            })
            tmp_df = pd.concat((market.to_frame().T, tmp_df), axis=1)
            tmp_df.insert(7, 'su', uuid)
        
            df = pd.concat((df, tmp_df), axis=0, ignore_index=True)
    return df

def run_convert_to_df(list_input, cores: int=2):
    with ThreadPool(cores) as pool:
        work_return = pool.map(_worker_convert_to_df, list_input)
    return work_return



In [ ]:
%%time
for index, df in list(enumerate(separation_frame(ret_df, 2)))[:1]:
    out = run_get_price(list(df.iterrows()), cores=100)
    out = run_convert_to_df(out, cores=50)
    # out = pd.c
    # _save_df_to_zip(out, f'result {index::03d}')

  0%|          | 0/103 [00:00<?, ?it/s]

In [ ]:
# pbar, (index, item) = pool_input
item = list(ret_df.iterrows())[1][1]
out_list = list()

res = loc_df[loc_df['slug'] == item.sity].min()
headers = {
            'x-locality-geoid': str(res.geoId),
            'x-position-latitude': f'{res.lat:.5f}',
            'x-position-longitude': f'{res.lng:.5f}'
            }
    
url = f'https://search.edadeal.io/api/v4/retailer/{item.data_uuid}/items'

params = {  'addContent': ['true'],
            'checkAdult': ['true'],
            'excludeSegmentSlug': ['alcohol', 'pt_alcool', 'en_alcohol', 'es_alcohol', 'tr_alkol'],
            'groupBy': ['sku_or_meta'],
            'numdoc': ['599'],
            'page': ['0'],
            'segmentUuid': []
         }

for su1 in seg_df['uuid'].unique()[:1]:
    params['segmentUuid'] = [su1]
    
    respons_1 = get_response(url, params, headers)
    
    if respons_1:
        tmp_list_01 = list()
        
        su2_list = seg_df[seg_df['uuid'] == su1]['uuid level 02'].unique()
        for su2 in su2_list[:]:
            params['segmentUuid'] = [su2]

            respons_2 = get_response(url, params, headers)
                            
            if respons_2:
                tmp_list_02 = list()
                try:
                    su3_list = list(set([(x.get('segmentUuids', []))[-1] for x in respons_2]))
                    len_sku_list = len(su3_list)
                except Exception:
                    su3_list = None
                
                if not su3_list or (len_sku_list<550):
                    print(su3_list, end='/')
                # su3_list = seg_df[(seg_df['uuid'] == su1) & (seg_df['uuid level 02'] == su2)]['uuid level 03'].unique()
                
                # for su3 in su3_list:
                #     params['segmentUuid'] = [su3]
                #     respons_3 = get_response(url, params, headers)

                #     if respons_3:
                #         tmp_list_02.append((su3, respons_3))
                
                # if tmp_list_02:
                #     tmp_list_01.extend(tmp_list_02)
                # else:
                #     tmp_list_01.append((su2, respons_2))

        # if tmp_list_01:
        #     out_list.extend(tmp_list_01)
        # else:
        #     out_list.append((su1, respons_1))